In [ ]:
#@title 🚀 Ailo Colab Miner v2.0 - Full Model Training!
#@markdown ### Instructions:
#@markdown 1. Runtime > Change runtime type > **T4 GPU**
#@markdown 2. Enter your wallet address below
#@markdown 3. Press **▶️ Play** - that's it!
#@markdown ---

WALLET = ""  #@param {type:"string"}

# ===== AILO COLAB MINER v2.0 - FULL MODEL =====
import subprocess, sys
subprocess.run([sys.executable, '-m', 'pip', 'install', 'torch', '--index-url', 'https://download.pytorch.org/whl/cu118', '-q'], capture_output=True)
subprocess.run([sys.executable, '-m', 'pip', 'install', 'aiohttp', 'requests', '-q'], capture_output=True)

import torch, torch.nn as nn, numpy as np, requests, aiohttp, asyncio, time, base64, gzip, gc, json, io

API, SERVER, VER = "https://ailo.site/api", "https://ailo.site", "2.0.0-colab-full"
MAX_LOSS = 2.8

if not torch.cuda.is_available():
    print("❌ GPU not enabled! Go to: Runtime > Change runtime type > T4 GPU")
    raise SystemExit

if len(WALLET) < 40:
    print("❌ Enter your wallet address above! Get one at: https://ailo.site/wallet.html")
    raise SystemExit

vram = torch.cuda.get_device_properties(0).total_memory/1e9
print(f"✅ GPU: {torch.cuda.get_device_name(0)} | VRAM: {vram:.1f}GB")
print(f"✅ Wallet: {WALLET[:12]}...")

class AILO1B(nn.Module):
    """AILO-1B: 899M parameter model"""
    def __init__(self):
        super().__init__()
        self.emb = nn.Embedding(50257, 1600)
        self.pos = nn.Parameter(torch.zeros(1, 512, 1600))
        self.tf = nn.TransformerEncoder(nn.TransformerEncoderLayer(1600, 25, 6400, 0.1, batch_first=True), 24)
        self.out = nn.Linear(1600, 50257)
    def forward(self, x):
        return self.out(self.tf(self.emb(x) * 40 + self.pos[:, :x.size(1)]))

class FullModelTrainer:
    """Trainer for full 899M model"""
    def __init__(self):
        self.dev = torch.device('cuda')
        self.m = None; self.opt = None; self.steps = 0; self.acc = 0
        self.best_loss = float('inf')
        self.saved_grads = {}  # All layer gradients
        self.synced = False
        self.layer_info = None
    
    def init(self):
        print("🧠 Loading AILO-1B (899M params)...")
        gc.collect(); torch.cuda.empty_cache()
        self.m = AILO1B().half().to(self.dev)
        params = sum(p.numel() for p in self.m.parameters())
        print(f"   ✅ Ready! {params:,} params | GPU Memory: {torch.cuda.memory_allocated()/1e9:.1f}GB")
        self.opt = torch.optim.AdamW(self.m.parameters(), lr=1e-4, weight_decay=0.01)
        self.crit = nn.CrossEntropyLoss()
    
    def sync_full_model(self):
        """Download and load full 1.8GB checkpoint"""
        try:
            print("🔄 Syncing full model (1.8GB)...")
            
            # Get model info
            r = requests.get(f"{SERVER}/api/model/weights", timeout=30)
            if r.status_code != 200: 
                print("   ⚠️ No model info available")
                return False
            
            data = r.json()
            server_loss = data.get('bestCheckpointLoss', 999)
            
            # Try full checkpoint first (1.8GB)
            print(f"   📥 Downloading full checkpoint (loss: {server_loss})...")
            cr = requests.get(f"{SERVER}/api/model/checkpoint/full", timeout=300, stream=True)
            
            if cr.status_code == 200:
                # Load full checkpoint
                weights = np.frombuffer(cr.content, dtype=np.float16)
                print(f"   📦 Loaded {len(weights):,} params ({len(weights)*2/1e9:.2f}GB)")
                
                # Get layer info
                info_r = requests.get(f"{SERVER}/api/model/checkpoint/full/info", timeout=10)
                if info_r.status_code == 200:
                    self.layer_info = info_r.json().get('layers', [])
                    
                    # Load each layer
                    with torch.no_grad():
                        for layer in self.layer_info:
                            name = layer['name']
                            offset = layer['offset']
                            size = layer['size']
                            shape = layer['shape']
                            
                            # Get the parameter
                            parts = name.split('.')
                            param = self.m
                            for p in parts:
                                if p.isdigit():
                                    param = param[int(p)]
                                else:
                                    param = getattr(param, p, None)
                                    if param is None: break
                            
                            if param is not None and hasattr(param, 'data'):
                                w = weights[offset:offset+size].astype(np.float32)
                                param.data = torch.from_numpy(w).reshape(shape).half().to(self.dev)
                    
                    self.synced = True
                    self.best_loss = float(server_loss) if isinstance(server_loss, (int, float)) else 2.8
                    print(f"   ✅ Full model loaded! Target loss: {self.best_loss:.4f}")
                    return True
            
            # Fallback to partial checkpoint
            print("   ⚠️ Full checkpoint not available, trying partial...")
            cr = requests.get(f"{SERVER}/api/model/checkpoint", timeout=60)
            if cr.status_code == 200:
                w = np.frombuffer(cr.content, dtype=np.float16).astype(np.float32)
                with torch.no_grad():
                    exp = self.m.out.weight.numel()
                    if len(w) >= exp:
                        self.m.out.weight.data = torch.from_numpy(w[:exp]).reshape_as(self.m.out.weight).half().to(self.dev)
                        self.synced = True
                        self.best_loss = float(server_loss) if isinstance(server_loss, (int, float)) else 2.8
                        print(f"   ✅ Partial model loaded (out.weight only)")
                return True
            return False
        except Exception as e:
            print(f"   ⚠️ Sync failed: {e}")
            return False
    
    def batch(self, w):
        try:
            r = requests.get(f"{API}/cuda/training-data", params={'batchSize': 1, 'wallet': w}, timeout=10)
            if r.ok:
                t = r.json().get('articles', [''])[0]
                if len(t) >= 33:
                    tk = [ord(c) % 50257 for c in t[:33]]
                    return torch.tensor([tk[:32]]), torch.tensor([tk[1:33]])
        except: pass
        return None, None
    
    def step(self, x, y):
        if x is None: return None
        self.m.train()
        x, y = x.to(self.dev), y.to(self.dev)
        with torch.cuda.amp.autocast():
            loss = self.crit(self.m(x).view(-1, 50257), y.view(-1)) / 16
        loss.backward()
        self.acc += 1
        if self.acc >= 16:
            nn.utils.clip_grad_norm_(self.m.parameters(), 1.0)
            # Save ALL gradients (sampled)
            for name, p in self.m.named_parameters():
                if p.grad is not None:
                    # Sample 1:10 to reduce size
                    g = p.grad.float().flatten()[::10].half().cpu()
                    self.saved_grads[name] = g
            self.opt.step(); self.opt.zero_grad(set_to_none=True); self.acc = 0
        self.steps += 1
        l = loss.item() * 16
        if l < self.best_loss and not np.isnan(l): self.best_loss = l
        if self.steps % 50 == 0: gc.collect(); torch.cuda.empty_cache()
        return l
    
    def get_all_gradients(self):
        """Get all gradients compressed"""
        if not self.saved_grads:
            print("   ⚠️ No gradients yet")
            return None
        gc.collect()
        
        # Concatenate all gradients
        all_grads = []
        grad_info = []
        for name, g in self.saved_grads.items():
            all_grads.append(g.numpy())
            grad_info.append({'name': name, 'size': len(g)})
        
        flat = np.concatenate(all_grads)
        print(f"   📊 Compressing {len(flat):,} gradient values...")
        
        # Compress
        comp = gzip.compress(flat.tobytes(), 4)
        
        return {
            'gradients': base64.b64encode(comp).decode(),
            'info': grad_info,
            'total_params': len(flat),
            'compressed_size': len(comp)
        }

async def hb(w, tps):
    try:
        async with aiohttp.ClientSession() as s:
            await s.post(f"{API}/cuda/register", json={'wallet': w, 'clientVersion': VER, 'deviceInfo': {'gpu_name': 'T4-Colab', 'vram_gb': 15, 'hashrate': tps, 'full_model': True}})
    except: pass

async def submit_full(w, grad_data, steps, loss):
    """Submit full model gradients"""
    try:
        payload = {
            'wallet': w,
            'gradients': grad_data['gradients'],
            'gradient_info': grad_data['info'],
            'epoch': steps,
            'loss': loss,
            'gpu': 'T4-Colab',
            'full_model': True,
            'total_params': grad_data['total_params']
        }
        async with aiohttp.ClientSession() as ss:
            async with ss.post(f"{API}/cuda/submit", json=payload, timeout=aiohttp.ClientTimeout(total=120)) as r:
                if r.status == 200: 
                    return (await r.json()).get('reward', 0)
    except Exception as e:
        print(f"   ⚠️ Submit error: {e}")
    return 0

async def run():
    print("=" * 50)
    print(f"  🚀 AILO-1B FULL MODEL Miner v{VER}")
    print("  📦 Training ALL 899M parameters!")
    print("=" * 50)
    
    await hb(WALLET, 0)
    t = FullModelTrainer(); t.init()
    
    if not t.sync_full_model():
        print("\n⚠️ Could not sync - will train from scratch")
    
    print("\n⛏️ FULL MODEL MINING STARTED!")
    print(f"📊 Dashboard: https://ailo.site/dashboard.html?wallet={WALLET[:12]}\n")
    
    rew, last_s, last_h, tps = 0.0, time.time(), time.time(), 0.0
    skipped = 0
    
    try:
        while True:
            t0 = time.time()
            x, y = t.batch(WALLET)
            if x is None:
                skipped += 1
                if skipped % 10 == 0: print(f"⏭️ Skipped {skipped} batches")
                await asyncio.sleep(0.5)
                continue
            
            loss = t.step(x, y)
            if loss is None: continue
            
            tps = 32 / (time.time() - t0 + 0.001)
            
            if time.time() - last_h >= 10: await hb(WALLET, tps); last_h = time.time()
            if t.steps % 25 == 0: 
                print(f"Step {t.steps} | Loss: {loss:.4f} | Best: {t.best_loss:.4f} | {tps:.0f} tok/s | 💰 {rew:.4f} ALC")
            
            # Submit every 5 minutes
            if time.time() - last_s >= 300:
                gc.collect(); torch.cuda.empty_cache()
                grad_data = t.get_all_gradients()
                
                if grad_data and t.synced and t.best_loss < MAX_LOSS and not np.isnan(t.best_loss):
                    print(f"\n📤 Submitting ALL gradients ({grad_data['compressed_size']/1e6:.1f}MB)...")
                    r = await submit_full(WALLET, grad_data, t.steps, t.best_loss)
                    rew += r
                    print(f"💰 +{r:.4f} ALC (Total: {rew:.4f})")
                elif not t.synced:
                    print("\n⏳ Not synced - training locally...")
                elif grad_data:
                    print(f"\n⏳ Loss too high ({t.best_loss:.4f})")
                
                t.sync_full_model()
                last_s = time.time()
                print()
                
    except KeyboardInterrupt: 
        print(f"\n⏹️ Stopped. Total earned: {rew:.4f} ALC")

await run()